In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pprint
import os
pp = pprint.PrettyPrinter(indent=4)

In [2]:
x_file, y_file = np.load("../data/x.npz"), np.load("../data/y.npz")
X = x_file['x']
Y = y_file['y_compress'].astype(np.int8)
print(f"X's shape is {X.shape}, Y's shape is {Y.shape}")
# _, inv_idx, path_count = np.unique(Y, return_inverse=True, return_counts=True)
# weight = path_count[inv_idx] / path_count.sum()

X's shape is (53022, 376), Y's shape is (53022, 2194)


In [3]:
# define the model
CONV_KERNEL_SZ = 9
CONV_STRIDE_SZ = 1
CONV_ACTVATION = "relu"
CONV_CHANNEL = 1
DROP_OUT_PROB = .25
DIM_X, DIM_Y = X.shape[1], Y.shape[1]

MODEL_SAVE_DIR = "../model/d2"
if os.path.exists(MODEL_SAVE_DIR):
    model = tf.keras.models.load_model(MODEL_SAVE_DIR)
else:
    model = keras.Sequential([
        # keras.layers.Conv1D(CONV_CHANNEL, CONV_KERNEL_SZ, strides=CONV_STRIDE_SZ, activation=CONV_ACTVATION, input_shape=(DIM_X, 1)),
        # keras.layers.Dropout(rate=DROP_OUT_PROB),    
        # keras.layers.Conv1D(CONV_CHANNEL, CONV_KERNEL_SZ, strides=CONV_STRIDE_SZ, activation=CONV_ACTVATION),      
        # keras.layers.Dropout(rate=DROP_OUT_PROB),
        # keras.layers.Conv1D(CONV_CHANNEL, CONV_KERNEL_SZ, strides=CONV_STRIDE_SZ, activation=CONV_ACTVATION),
        # keras.layers.Flatten(),  
        # # keras.layers.Dense(4096),
        keras.layers.Input(DIM_X),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(DIM_Y, activation='sigmoid')
    ])
# model = keras.Sequential([
#     keras.layers.Dense(DIM_Y, activation='relu', input_dim=DIM_X),
#     keras.layers.Dense(DIM_Y, activation='relu'),
# ])
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['binary_accuracy', keras.metrics.Recall(thresholds=0.5)])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               37700     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2194)              221594    
Total params: 269,394
Trainable params: 269,394
Non-trainable params: 0
_________________________________________________________________


In [4]:
X_expand = X.astype(np.float32)
# X_expand = np.expand_dims(X_expand.astype(np.float32), axis=-1)
data = tf.data.Dataset.from_tensor_slices((X_expand, Y.astype(np.int32)))

In [5]:
data.shuffle(X.shape[0])
n_train = int(X.shape[0] * 0.75)
train, test = data.take(n_train), data.skip(n_train)   # 75% split

In [6]:
train = train.batch(100)
valid_data = test
valid_data = valid_data.batch(100)
train.element_spec, valid_data.element_spec

((TensorSpec(shape=(None, 376), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 2194), dtype=tf.int32, name=None)),
 (TensorSpec(shape=(None, 376), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 2194), dtype=tf.int32, name=None)))

In [7]:
model.reset_states()
history = model.fit(
    train,
    epochs=20,
    validation_data=valid_data
)

Epoch 1/20
398/398 [==============================] - 7s 15ms/step - loss: 0.6705 - binary_accuracy: 0.7323 - recall: 0.7126 - val_loss: 0.4302 - val_binary_accuracy: 0.7857 - val_recall: 0.7322
Epoch 2/20
398/398 [==============================] - 6s 14ms/step - loss: 0.4341 - binary_accuracy: 0.7776 - recall: 0.7299 - val_loss: 0.4046 - val_binary_accuracy: 0.8366 - val_recall: 0.7491
Epoch 3/20
398/398 [==============================] - 6s 14ms/step - loss: 0.4228 - binary_accuracy: 0.7763 - recall: 0.7114 - val_loss: 0.3803 - val_binary_accuracy: 0.8501 - val_recall: 0.7553
Epoch 4/20
398/398 [==============================] - 6s 14ms/step - loss: 0.4203 - binary_accuracy: 0.7753 - recall: 0.7106 - val_loss: 0.3781 - val_binary_accuracy: 0.8457 - val_recall: 0.7643
Epoch 5/20
398/398 [==============================] - 6s 14ms/step - loss: 0.4183 - binary_accuracy: 0.7724 - recall: 0.7089 - val_loss: 0.3798 - val_binary_accuracy: 0.8427 - val_recall: 0.7689
Epoch 6/20
398/398 [=====

In [8]:
model.save(MODEL_SAVE_DIR)

INFO:tensorflow:Assets written to: ../model/d2\assets


In [9]:
test = test.batch(100)
eval_results = model.evaluate(test)

133/133 [==============================] - 2s 9ms/step - loss: 0.3684 - binary_accuracy: 0.8592 - recall: 0.7473
